In [56]:
# Finally, We are on #5 MEMORY! Let's goooo

# If you don't add 'memory' to chatbot, chatbot can't remember anything..
# even following questions, chatbot dont understand if dont have memory => stateless

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"},{"output": "How are you?"})

memory.load_memory_variables({})

# ConversationBufferMemory save all conversation. it's inefficient. 

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [57]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})


{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [58]:
add_message(5,5)
memory.load_memory_variables({})
# ConversationBufferWindowMemory fix conversation size always. it's good to make efficient, but chatbot cant remember old messages..

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [59]:
# now, we use llm with memory. it costs.
# ConversationSummaryMemory: summary of the conversation automatically
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def get_history():
    return memory.load_memory_variables({})

add_message("Hi, My name is Sofia Kim. i live in South Korea", "Wow that is so cool")

In [60]:
add_message("Korea is pretty", "I wish to go there")

get_history()

{'history': 'The human introduces herself as Sofia Kim from South Korea. The AI responds with enthusiasm about her location and name, expressing a desire to visit Korea because it is pretty.'}

In [61]:
# ConversationSummaryBufferMemory = window buffer memory + buffer window memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)
# max_token_limit is max size before summery (original conversation max)

add_message("Hi, My name is Sofia Kim. i live in Japan", "Wow that is so cool")
get_history()

{'history': [HumanMessage(content='Hi, My name is Sofia Kim. i live in Japan'),
  AIMessage(content='Wow that is so cool')]}

In [62]:
add_message("Japan weather is warm", "I wish I could go!")
get_history()

{'history': [HumanMessage(content='Hi, My name is Sofia Kim. i live in Japan'),
  AIMessage(content='Wow that is so cool'),
  HumanMessage(content='Japan weather is warm'),
  AIMessage(content='I wish I could go!')]}

In [63]:
add_message("How far Japan to Korea?", "near")
get_history()

{'history': [SystemMessage(content='The human introduces herself as Sofia Kim and mentions she lives in Japan.'),
  AIMessage(content='Wow that is so cool'),
  HumanMessage(content='Japan weather is warm'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far Japan to Korea?'),
  AIMessage(content='near')]}

In [64]:
add_message("Korea dish is so delicious", "Oh I like Korean Food!")
get_history()

{'history': [SystemMessage(content='The human introduces herself as Sofia Kim and mentions she lives in Japan. The AI expresses admiration for this and Sofia mentions the warm weather in Japan, prompting the AI to express a desire to visit.'),
  HumanMessage(content='How far Japan to Korea?'),
  AIMessage(content='near'),
  HumanMessage(content='Korea dish is so delicious'),
  AIMessage(content='Oh I like Korean Food!')]}

In [65]:
# ConversationKGMemory : it choose/extract 'entity' from your conversation
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

add_message("Hi, My name is Sofia Kim. i live in Japan", "Wow that is so cool")

In [66]:
memory.load_memory_variables({"input": "who is Sofia Kim"})

{'history': [SystemMessage(content='On Sofia Kim: Sofia Kim lives in Japan.')]}

In [67]:
add_message("Sofia likes Kimchi", "Wow that is so cool")
memory.load_memory_variables({"input": "What does she like"})

{'history': [SystemMessage(content='On Sofia: Sofia likes Kimchi.')]}

In [68]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

# we need 'space' for memory. that is 'chat_history'(variable)
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=PromptTemplate.from_template(template)
)

chain.predict(question="My name is Sofia Kim")
chain.predict(question="I live in Seoul. it's beautiful city")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Sofia Kim
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Sofia Kim
AI: Hello Sofia Kim! How can I assist you today?
    Human:I live in Seoul. it's beautiful city
    You:


> Finished chain.


"That's wonderful to hear! Seoul is indeed a beautiful city with a rich history and vibrant culture. Is there anything specific you would like to know or discuss about Seoul?"

In [69]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Sofia Kim
AI: Hello Sofia Kim! How can I assist you today?
Human: I live in Seoul. it's beautiful city
AI: That's wonderful to hear! Seoul is indeed a beautiful city with a rich history and vibrant culture. Is there anything specific you would like to know or discuss about Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Sofia Kim.'

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
# memory class can out two types. 
# One type is string, and One type is message
memory.load_memory_variables({}) # you can see print just 'text'(string)

# If you change output type to message, follow below parameter : return_message
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=prompt
)


chain.predict(question="My name is Sofia Kim") # You can see print type 'message'
chain.predict(question="I live in Seoul. it's beautiful city")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human.
Human: My name is Sofia Kim

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human.
Human: My name is Sofia Kim
AI: Nice to meet you, Sofia Kim! How can I assist you today?
Human: I live in Seoul. it's beautiful city

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: you are a helpful AI talking to a human.
Human: My name is Sofia Kim
AI: Nice to meet you, Sofia Kim! How can I assist you today?
Human: I live in Seoul. it's beautiful city
AI: Seoul is indeed a beautiful city with a rich history and vibrant culture. Is there anything specific you enjoy about living in Seoul?
Human: What is my name?

> Finished chain.


'Your name is Sofia Kim.'

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(_): 
    # if you dont put any parameter, it will cause error. 
    # because it should given 'question'(=input). every memory class given input, and through out output. remember! 
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm # you can use several functions in RunnablePassthrough class

def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    memory.save_context({"input": question},{"output": result.content})
    print(result)

invoke_chain("My name is Sofia")
invoke_chain("What is my name?")

# So far, we learned 3 ways to add 'memory' in 'prompt'
# 1. using 'LLM chain'
# 2. using 'Chat prompt template'
# 3. using 'manual memory management' (it's not auto, but this way could be better and efficient)

content='Nice to meet you, Sofia! How can I assist you today?'
content='Your name is Sofia.'


In [86]:
# RAG(Retrieval Augmented Generation)
# your own data + general data = prompt => model

# Retrival : Langchain module
# Retriving data course : source data => load data => split data (transform) / => embed (talk about later) => store
# loader : extract data from source and give them to langchain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# UnstructuredFileLoader can load any type of file
loader = UnstructuredFileLoader("./files/chapter_one.txt")


# It's too big file, we need to split out. there is 2 ways
# 1.
# docs = loader.load()
# splitter.split_documents(docs)
# 2.
loader.load_and_split(text_splitter=splitter)
len(loader.load_and_split(text_splitter=splitter))


Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

45